In [ ]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
from glob import glob, iglob
from tqdm import tqdm
import matplotlib.pyplot as plt

import numpy as np; print('numpy', np.__version__)
import cv2; print('opencv', cv2.__version__)
import sklearn; print('sklearn', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import tensorflow.keras as keras; print('keras', keras.__version__)

import settings
import helper

In [ ]:
sys.path.extend(['./models'])
print('sys.path', sys.path)

In [ ]:
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()

# gpu allow_growth
config.gpu_options.allow_growth = True
keras.backend.set_session(tf.Session(config=config))

In [ ]:
model_name = 'densenet3d'
wtype='lung'
dataset_name = 'cube_'+wtype
unique_id = int(time.time())

img_slice, img_rows, img_cols, channels = 32, 32, 32, 1
num_classes = 3

batch_size = 64
epochs = 999

In [ ]:
from models.se_densenet_3d import SEDenseNet

model = SEDenseNet(input_shape=(img_slice, img_rows, img_cols, channels),
                        depth=52,
                        nb_dense_block=4,
                        growth_rate=12,
                        nb_filter=-1,
                        nb_layers_per_block=-1,
                        bottleneck=True,
                        reduction=0.5,
                        dropout_rate=0.2,
                        weight_decay=1e-4,
                        subsample_initial_block=True,
                        maxpool_initial_block=False,
                        include_top=True,
                        weights=None,
                        input_tensor=None,
                        classes=num_classes,
                        activation='softmax')

model.summary()
                                                                          

In [ ]:
print(model_name, len([l.name for l in model.layers if ('conv' in str(type(l)) or 'dense' in str(type(l)))])+1)

In [ ]:
from tensorflow.keras.models import load_model

model = load_model(f'./output/{model_name}/1564138475/' + "model-best.hd5")

In [ ]:
from generator import get_classify_file_list, split_dataset, ClassifySequence
dataset = get_classify_file_list(wtype=wtype)

trainset, validset, _ = split_dataset(dataset, train_by_valid=0.02)

gen_train = ClassifySequence(trainset, wtype=wtype, batch_size=batch_size)
gen_valid = ClassifySequence(validset, wtype=wtype, batch_size=batch_size)

print(len(gen_train), len(gen_valid))


In [ ]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

output_dir = f'./output/{model_name}/{unique_id}/'
print('output:', output_dir)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

checkpoint = ModelCheckpoint(filepath=output_dir + "weights-e{epoch:03d}-{acc:.4f}.hd5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=True, 
                             mode='auto', 
                             save_freq='epoch',
                             load_weights_on_restart=False)

checkpoint_best = ModelCheckpoint(filepath=output_dir + "model-best.hd5", 
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only=True, 
                                  save_weights_only=False,
                                  mode='auto', 
                                  save_freq='epoch',
                                  load_weights_on_restart=False)

earlyStopping = EarlyStopping(monitor='val_loss', 
                              min_delta=0, 
                              patience=12, 
                              verbose=1, 
                              mode='auto', 
                              baseline=None, 
                              restore_best_weights=False)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=1e-8, cooldown=0, min_lr=0)
tensorBoard = TensorBoard(log_dir=output_dir + 'logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
csvLogger = CSVLogger(filename=output_dir + "train_logs.csv", separator=',', append=False)

# model.fit(X_train, Y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           shuffle=True,
#           verbose=1,
#           validation_data=(X_valid, Y_valid),
#           callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
#           )

In [ ]:
model.fit_generator(generator=gen_train,
                    steps_per_epoch=len(gen_train), 
                    epochs=epochs, 
                    verbose=1, 
                    callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger], 
                    validation_data=gen_valid, 
                    validation_steps=len(gen_valid), 
                    validation_freq=1, 
                    class_weight=None, 
                    max_queue_size=10, 
                    workers=1, 
                    use_multiprocessing=False, 
                    shuffle=True, 
                    initial_epoch=0)
